<a href="https://colab.research.google.com/github/aulialigar/computer-vision/blob/main/TKC_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/TKC')
!ls -a

Mounted at /content/drive
1  2  3


In [2]:
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
base_dir = '/content/drive/My Drive/TKC'

In [4]:
IMAGE_SIZE = [150, 150]

In [5]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'wrap',
                    validation_split=0.1)
 
validation_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'wrap',
                    validation_split=0.1)

In [6]:
# persiapan data training
train_generator = train_datagen.flow_from_directory(
        base_dir,  
        # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        target_size=IMAGE_SIZE,
        shuffle = True,
        subset='training',
        class_mode='categorical')

# persiapan data validasi
validation_generator = validation_datagen.flow_from_directory(
        base_dir, 
        # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        target_size=IMAGE_SIZE,
        subset='validation',
        class_mode='categorical')

Found 2768 images belonging to 3 classes.
Found 305 images belonging to 3 classes.


In [7]:
len(train_generator)
len(validation_generator)

10

In [8]:
pre_trained_model = tf.keras.applications.VGG16(input_shape=(150, 150, 3), include_top=False, weights="imagenet")

58900480/58889256 [==============================] - 1s 0us/step


In [9]:
for layer in pre_trained_model.layers:
  print(layer.name)
  layer.trainable = False

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [10]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(3, activation='softmax')(x)

In [11]:
model = tf.keras.Model(pre_trained_model.input, x)

In [12]:
model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])

In [13]:
vgg_classifier = model.fit(train_generator,
    validation_data = validation_generator,
    epochs = 100)

Epoch 1/100
87/87 [==============================] - 873s 10s/step - loss: 0.7966 - acc: 0.6568 - val_loss: 0.5154 - val_acc: 0.8033
Epoch 2/100
87/87 [==============================] - 63s 729ms/step - loss: 0.5228 - acc: 0.7691 - val_loss: 0.7022 - val_acc: 0.6656
Epoch 3/100
87/87 [==============================] - 62s 715ms/step - loss: 0.4556 - acc: 0.8053 - val_loss: 0.5210 - val_acc: 0.7836
Epoch 4/100
87/87 [==============================] - 62s 713ms/step - loss: 0.4522 - acc: 0.8111 - val_loss: 0.5205 - val_acc: 0.7738
Epoch 5/100
87/87 [==============================] - 62s 718ms/step - loss: 0.4172 - acc: 0.8183 - val_loss: 0.4670 - val_acc: 0.8000
Epoch 6/100
87/87 [==============================] - 63s 728ms/step - loss: 0.4102 - acc: 0.8316 - val_loss: 0.5983 - val_acc: 0.7410
Epoch 7/100
87/87 [==============================] - 63s 727ms/step - loss: 0.4124 - acc: 0.8208 - val_loss: 0.3924 - val_acc: 0.8262
Epoch 8/100
87/87 [==============================] - 63s 721ms/

In [17]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [18]:
def evaluate(model):

  validation_generator = validation_datagen.flow_from_directory(
          base_dir, 
          target_size=(150,150), 
          batch_size=32, 
          class_mode='categorical',
          shuffle = False,
          subset='validation')

  batch_size = 32
  num_of_test_samples = len(validation_generator.filenames)

  Y_pred = model.predict_generator(validation_generator, num_of_test_samples // batch_size+1)
  y_pred = np.argmax(Y_pred, axis=1)

  print('\nConfusion Matrix\n')
  print(confusion_matrix(validation_generator.classes, y_pred))


  print('\n\nClassification Report\n')
  target_names = ['1', '2', '3']
  print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [19]:
evaluate(model)

Found 305 images belonging to 3 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  



Confusion Matrix

[[ 33  34   3]
 [ 13 129   0]
 [  3   4  86]]


Classification Report

              precision    recall  f1-score   support

           1       0.67      0.47      0.55        70
           2       0.77      0.91      0.83       142
           3       0.97      0.92      0.95        93

    accuracy                           0.81       305
   macro avg       0.80      0.77      0.78       305
weighted avg       0.81      0.81      0.80       305



In [20]:
model.save('vgg.h5')

In [ ]:
# https://medium.com/nerd-for-tech/image-classification-using-transfer-learning-vgg-16-2dc2221be34c